In [24]:
#importing packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
import numpy as np
import os
%matplotlib inline

In [25]:
def read_ip_image(path):
    # Reading input image
    ip_img = cv.imread(path)
    return ip_img

In [26]:
def GrayScale(ip_img):
    # Converting input color image to gray
    gray = cv.cvtColor(ip_img, cv.COLOR_RGB2GRAY)
    return gray

def Gaussian(gray, kernel_size):
    #Smoothing image using Gaussian Smoothin
    kernel_size = 5
    blur_gray = cv.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    return blur_gray


In [27]:
def CannyEdge(blur_gray, low_threshold, high_threshold):
    # Edge detection using Canny filter
    edges = cv.Canny(blur_gray, lw_threshold, hg_threshold)
    return edges


In [28]:
def region_of_interest(edges, vertices):
    # Creating masked edges 
    mask = np.zeros_like(edges)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(edges.shape) > 2:
        channel_count = edges.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (240,) * channel_count
    else:
        ignore_mask_color = 240
   
    cv.fillPoly(mask, vertices, ignore_mask_color)
    
    masked_edges = cv.bitwise_and(edges, mask)
    
    return masked_edges

In [29]:
def hough_lines(img, edges, masked_edges, rho, theta, threshold, min_line_length, max_line_gap):

    # Creating blank image of shape ip image
    line_image = np.copy(img)*0
    
    #plt.figure()
    #plt.imshow(line_image)
    
    # Run Hough on edge detected image
    lines = cv.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    #line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    # Seperating left and right lane coordinates
    left_lane_pts = []
    right_lane_pts = []
            
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 <= 460:
                left_lane_pts.append(line)
            else:
                right_lane_pts.append(line)

    
    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges))

    # Draw the lines on the edege image
    lines_edges = cv.addWeighted(color_edges, 1, line_image, 0.6, 0)
    
    draw_lines(img, left_lane_pts)
    draw_lines(img, right_lane_pts)

    #plt.figure()
    #plt.imshow(img)
    return img

In [57]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    # Connecting dotted lanes markings 
    # Felt that its not need to compute slope and intrapolate/extrapolate the points 
    # to fill the dotted lanes
   
    for n in range(len(lines)-1):
        p1 = lines[n]
        p2 = lines[n+1]
        cv.line(img, (p1[0][2], p1[0][3]), (p2[0][0], p2[0][1]), (255, 0, 0), 2)
            
    # Tried below extrapolating logic, but did not work
    # Extrapolated points to finish the line_draw to [x, 540] coordinate
    # s = (y2 - y1)/(x2 - x1)
    # We know Slope, x1, y1 (any know point), we know y2 i.e 540, unknown is x2
    # x2 = [(y2 - y1)/m] + x1
    #slope = int((p2[0][1] - p1[0][1])/(p2[0][0] - p1[0][0]))
        
    #if ((p2[0][0] - p1[0][0]) > 0):
        #x2 = int((540 - p1[0][1])/int(slope)) + p1[0][0]
        #cv.line(img, (int(x2), 540), (p1[0][0], p1[0][1]), (255, 0, 0), 2)
        #cv.line(img, (p1[0][0], 540), (int(x2), p1[0][1]), (255, 0, 0), 2)
    


In [58]:
def weighted_img(line_img, color_edges, alpha=0.8, beta=1, y=0):
    return cv.addWeighted(color_edges, alpha, line_img, beta, y)

In [59]:
def process_image(img):
    global kernel_size, lw_threshold, hg_threshold
    global rho, theta, threshold, min_line_length, max_line_gap

    line_image = []
    gray = blur_gray = []
    masked_edges = initial_img = line_image = []

    gray = cv.cvtColor(img, cv.COLOR_RGB2YUV)
    
    # Making copy if img
    initial_img = img
        
    # Reading image shape
    imshape = img.shape
        
    # Selecting Polygon mask region
    vertices = np.array([[(60, imshape[0]), (460,320), (imshape[1]-460, 315), (imshape[1]-60, imshape[0])]], dtype=np.int32)
    
    # Converting input color image intto gray image
    gray = GrayScale(img)
        
    # Gaussian smoothing gray image
    blur_gray = Gaussian(gray, kernel_size)
        
    # Running Canny edge detector on blur_img
    edges = CannyEdge(blur_gray, lw_threshold, hg_threshold)
        
    # Selecting region of interest
    masked_edges = region_of_interest(edges, vertices)
    
            
    line_image = hough_lines(img, edges, masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    return line_image
    

## main()

In [61]:
# Gaussian filter, kernel size
kernel_size = 17
    
# Threshold values for Canny Edge detector
lw_threshold = 50
#hg_threshold = 150
hg_threshold = 100
    
# Hough transform parameters
rho = 2
theta = np.pi/180
threshold = 5
min_line_length = 15
max_line_gap = 5
    
    
if __name__ == "__main__":

    global kernel_size, lw_threshold, hg_threshold
    global rho, theta, threshold, min_line_length, max_line_gap
    
    # Input test image directory
    ip_dir = os.listdir("test_images/")

    # Loop to traverse of list of input image files
    for fpath in ip_dir:
        
        # Intializing numpy arrays
        line_image = []
        gray = blur_gray = []
        masked_edges = initial_img = line_image = []
        
        img = read_ip_image("test_images/" + str(fpath))
            
        if img is None:
            print fpath, "is not imge fie"
            continue
        
        # Making copy if img
        initial_img = img
        
        # Reading image shape
        imshape = img.shape
        
        # Selecting Polygon mask region
        vertices = np.array([[(120, imshape[0]), (460,320), (imshape[1]-460, 315), (imshape[1]-90, imshape[0])]], dtype=np.int32)
    
    
        # Converting input color image intto gray image
        gray = GrayScale(img)
        
        # Gaussian smoothing gray image
        blur_gray = Gaussian(gray, kernel_size)
        
        # Running Canny edge detector on blur_img
        edges = CannyEdge(blur_gray, lw_threshold, hg_threshold)
        
        # Selecting region of interest
        masked_edges = region_of_interest(edges, vertices)
            
        line_image = hough_lines(img, edges, masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
        lines_edge = weighted_img(line_image, initial_img)
        
        cv.imwrite("test_out_images/"+str(fpath), lines_edge)
        
        #plt.figure()
        #plt.imshow(lines_edge)
    print "Done running test_images...!!!"
    print "Check output images under test_out_images\..!!!"
    
    # video file as input
    # Import everything needed to edit/save/watch video clips
    from moviepy.editor import VideoFileClip
    from IPython.display import HTML

    ip_dir = os.listdir("test_videos/")
    
    
    for video_ip in ip_dir:
        new_clip_output = "test_out_videos/"+str(video_ip)
        clip1 = VideoFileClip("test_videos/"+str(video_ip)).subclip(1,3)
        new_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
        %time new_clip.write_videofile(new_clip_output, audio=False)    
    
    print "Output Video files written to \\test_out_videos dir..!!!"


Done running test_images...!!!
Check output images under test_out_images\..!!!
[MoviePy] >>>> Building video test_out_videos/solidWhiteRight.mp4
[MoviePy] Writing video test_out_videos/solidWhiteRight.mp4


 98%|█████████▊| 50/51 [00:03<00:00,  7.57it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_out_videos/solidWhiteRight.mp4 

CPU times: user 836 ms, sys: 40 ms, total: 876 ms
Wall time: 5.99 s
[MoviePy] >>>> Building video test_out_videos/challenge.mp4
[MoviePy] Writing video test_out_videos/challenge.mp4


 98%|█████████▊| 50/51 [00:07<00:00,  4.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_out_videos/challenge.mp4 

CPU times: user 1.79 s, sys: 280 ms, total: 2.07 s
Wall time: 17 s
[MoviePy] >>>> Building video test_out_videos/solidYellowLeft.mp4
[MoviePy] Writing video test_out_videos/solidYellowLeft.mp4


 98%|█████████▊| 50/51 [00:04<00:00,  8.10it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_out_videos/solidYellowLeft.mp4 

CPU times: user 796 ms, sys: 84 ms, total: 880 ms
Wall time: 6.62 s
Output Video files written to \test_out_videos dir..!!!
